# Meraki Python SDK Demo: SSID Limits Checker

*This notebook demonstrates using the Meraki Python SDK to check all SSIDs in a network for bandwidth limits. If any are found, it also provides the option to remove them all automatically.*

If you have users complaining about slow WiFi, you might like to check if there are any SSID-wide speed limits. With the Meraki Dashboard API, its SDK and Python, we can check for speed limits on any/all SSIDs across the organization without digging through the GUI.

---

>NB: Throughout this notebook, we will print values for demonstration purposes. In a production Python script, the coder would likely remove these print statements to clean up the console output.

In this first cell, we install and import the required meraki and os modules, and open the Dashboard API connection using the SDK. Make sure you have [set up your environment variables first](https://github.com/meraki/dashboard-api-python/blob/master/notebooks/notebooksReadme.md#setting-up-your-environment-variables).

In [ ]:
# Install the relevant modules. If you are using a local editor (e.g. VS Code, rather than Colab) you can run these commands, without the preceding %, via a terminal. NB: Run `pip install meraki==` to find the latest version of the Meraki SDK.
%pip install meraki
%pip install tablib

# If you are using Google Colab, please ensure you have set up your environment variables as linked above, then delete the two lines of ''' to activate the following code:
'''
%pip install colab-env -qU
import colab_env
'''

# Rely on meraki SDK, os, and tablib -- more on tablib later
import meraki
import os
import tablib

# We're also going to import Python's built-in JSON module, but only to make the console output pretty. In production, you wouldn't need any of the printing calls at all, nor this import!
import json

# Setting API key this way, and storing it in the env variables, lets us keep the sensitive API key out of the script itself
# The meraki.DashboardAPI() method does not require explicitly passing this value; it will check the environment for a variable
# called 'MERAKI_DASHBOARD_API_KEY' on its own. In this case, API_KEY is shown simply as an reference to where that information is
# stored.
API_KEY = os.getenv('MERAKI_DASHBOARD_API_KEY')

# Initialize the Dashboard connection.
dashboard = meraki.DashboardAPI()

Let's make a basic pretty print formatter, `printj()`. It will make reading the JSON later a lot easier, but won't be necessary in production scripts.

In [ ]:
def printj(ugly_json_object):
    
    # The json.dumps() method converts a JSON object into human-friendly formatted text
    pretty_json_string = json.dumps(ugly_json_object, indent = 2, sort_keys = False)

    return print(pretty_json_string)

Most API calls require passing values for the organization ID and/or the network ID. In this second cell, we fetch a list of the organizations the API key can access, then pick the first org in the list, and the first network in that organization, to use for later operations. You could re-use this code presuming your API key only has access to a single organization, and that organization only contains a single network. Otherwise, you would want to review the organizations object declared and printed here to review its contents.

In [ ]:
# Let's make it easier to call this data later
# getOrganizations will return all orgs to which the supplied API key has access
organizations = dashboard.organizations.getOrganizations()
print('Organizations:')
printj(organizations)

# This example presumes we want to use the first organization as the scope for later operations. 
firstOrganizationId = organizations[0]['id']
firstOrganizationName = organizations[0]['name']

# Print a blank line for legibility before showing the firstOrganizationId
print('')
print(f'The firstOrganizationId is {firstOrganizationId}, and its name is {firstOrganizationName}.')

This example will analyze and potentially change every SSID in every network in your organization. It is fine to re-use presuming that that's what you want to do. Otherwise, you might want to review the `networks` list and operate on just one of them instead.

In [ ]:
networks = dashboard.organizations.getOrganizationNetworks(organizationId=firstOrganizationId)
print('Networks:')
printj(networks)

Now that we've got the organization and network values figured out, we can get to the ask at hand:

> Check for any SSID-level bandwidth limits.

We can only run this on networks that have wireless devices, so we have a `for` loop that checks each entry in the `networks` list. If the network's `productTypes` value contains `wireless`, then we'll pull the SSIDs from it.

The `getNetworkWirelessSsids` endpoint will return the SSIDs (enabled or otherwise, with or without limits) for the network. We will use a [list comprehension](https://www.datacamp.com/community/tutorials/python-list-comprehension) to make a new list, `organization_ssids_with_limits`, that contains any with a bandwidth limit set.

>NB: There are also traffic-shaping rules that are applied on a per-rule basis. This part does not 


In [ ]:
# Create an empty list where we can store all of the organization's SSIDs
organization_ssids = []

# Let's make a list of all the organization's SSIDs
for network in networks:
    # We only want to examine networks that might contain APs
    if 'wireless' in network['productTypes']:
        # let's find every SSID
        for ssid in dashboard.wireless.getNetworkWirelessSsids(network['id']):
            # Add each network's SSIDs to organization_ssids
            organization_ssids.append({'networkId': network['id'], 'ssid': ssid})




In [ ]:
# Let's make a list of organization SSIDs that have SSID-wide bandwidth limits set
organization_ssids_with_limits = [
    {'networkId': i['networkId'], 'number': i['ssid']['number']} for i in organization_ssids 
    if i['ssid']['perClientBandwidthLimitUp'] 
    or i['ssid']['perClientBandwidthLimitDown'] 
    or i['ssid']['perSsidBandwidthLimitUp'] 
    or i['ssid']['perSsidBandwidthLimitDown']
]

# Let's inform the user what we found
if len(organization_ssids_with_limits):
    print('These SSIDs have bandwidth limits:')
    printj(organization_ssids_with_limits)
else:
    print('There are no SSIDs with bandwidth limits set on the SSID level.')

To remove the SSID limits, we will modify these values:
* On the SSID level, using `updateNetworkWirelessSsid`, we will set all bandwidth limits to 0, which is "unlimited."
* Separately, we will remove any custom traffic shaping rules using `updateNetworkWirelessSsidTrafficShapingRules`.

Let's write a method that we can call later to do this:

In [ ]:
# Let's create a function that removes any found limits. We might use this later.
def removeSsidLimits(ssids):
	for ssid in ssids:
		# Remove SSID-wide limits
		dashboard.wireless.updateNetworkWirelessSsid(
			ssid['networkId'],
			ssid['number'],
			perClientBandwidthLimitUp=0,
			perClientBandwidthLimitDown=0,
			perSsidBandwidthLimitUp=0,
			perSsidBandwidthLimitDown=0
			)
		
		# Disable rule-based traffic-shaping rules
		dashboard.wireless.updateNetworkWirelessSsidTrafficShapingRules(
			ssid['networkId'],
			ssid['number'],
			rules=[]
		)

We will also define a separate method that removes custom traffic shaping rules everywhere, using the same `updateNetworkWirelessSsidTrafficShapingRules` method we used above, but this time applying it to all SSIDs. Like before, we're defining the method here, and we'll give the user the option to run this later.

In [ ]:
def removeCustomTrafficShapingRules():
    # We'll check each network
    for network in networks:
        # We only want to examine networks that might contain APs
        if 'wireless' in network['productTypes']:
            # SSIDs are always numbered 1-15 (0-14 in the API)
            for ssidNumber in range(15):
                # Disable rule-based traffic shaping for that network's SSID
                dashboard.wireless.updateNetworkWirelessSsidTrafficShapingRules(
                    network['id'],
                    ssidNumber,
                    rules=[]
                )

# Final steps

Here we're going to give the user an interactive prompt. First we set a few string literals that we can reuse to keep the code tight, then we call `removeSsidLimits` on `organization_ssids_with_limits` if the user confirms the appropriate prompts. **Use with care--there's no undo!**

In [ ]:
# Re-used strings
CONFIRM_STRING = 'OK, are you sure you want to do this? This script does not have an "undo" feature.'
CANCEL_STRING = 'OK. Operation canceled.'
WORKING_STRING = 'Working...'
COMPLETE_STRING = 'Operation complete.'

# Let's give the user the option to clear those bandwidth limits
if len(organization_ssids_with_limits):
    print('Would you like to remove all SSID-level bandwidth limits?')
    if input('([Y]es/[N]o):') in ['Y', 'y', 'Yes', 'yes', 'ye', 'Ye']:
        print(CONFIRM_STRING)
        if input('([Y]es/[N]o):') in ['Y', 'y', 'Yes', 'yes', 'ye', 'Ye']:
            print(WORKING_STRING)
            removeSsidLimits(organization_ssids_with_limits)
            print(COMPLETE_STRING)
        else:
            print(CANCEL_STRING)
    else:
        print(CANCEL_STRING)

As one last option, we can also mass-remove custom traffic shaping rules from all SSIDs across all organizations. This might be useful if, in the past, an admin had set a custom traffic shaping rule, but it's unclear where it was set. **Use with care--there's no undo!**

In [ ]:
# Let's also check if the user wants to take the extra step to remove all rule-based limits
print('There may also be client bandwidth limits on custom traffic shaping rules. Would you also like to remove any and all custom traffic shaping rules? This may take some time depending on the size and quantity of your networks. This will not clear default traffic shaping rules.')
if input('([Y]es/[N]o):') in ['Y', 'y', 'Yes', 'yes', 'ye', 'Ye']:
    print(CONFIRM_STRING)
    if input('([Y]es/[N]o):') in ['Y', 'y', 'Yes', 'yes', 'ye', 'Ye']:
        print(WORKING_STRING)
        removeCustomTrafficShapingRules()
        print(COMPLETE_STRING)
    else:
        print(CANCEL_STRING)
else:
    print(CANCEL_STRING)
        

# Final thoughts

And we're done! Hopefully you found this a useful demonstration of just a few things that are possible with Meraki's Python SDK. These additional resources may prove useful along the way.

[Meraki Interactive API Docs](https://developer.cisco.com/meraki/api-v1/#!overview): The official (and interactive!) Meraki API and SDK documentation repository on DevNet.

[VS Code](https://code.visualstudio.com/): An excellent code editor with full support for Python and Python notebooks.

[Automate the Boring Stuff with Python](https://automatetheboringstuff.com/): An excellent learning resource that puts the real-world problem first, then teaches you the Pythonic solution along the way.